In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tfidf_chatbot import FAQChatbot  # si tienes el módulo en la misma carpeta

df = pd.read_csv("wut_faq_dataset_bilingual.csv")

questions = df["question"].values
answers = df["answer"].values
categories = df["category"].values
languages = df["language"].values
source_urls = df["source_url"].values


In [3]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    stop_words="english",
    max_features=1000
)
question_vectors = vectorizer.fit_transform(questions)

chatbot = FAQChatbot(
    vectorizer=vectorizer,
    question_vectors=question_vectors,
    questions=questions,
    answers=answers,
    categories=categories,
    source_urls=source_urls,
    languages=languages,
)
print("TF-IDF chatbot ready. FAQ size:", len(questions))


TF-IDF chatbot ready. FAQ size: 44


In [4]:
test_queries = [
    "What scholarships can I get?",
    "How do I apply for admission?",
    "When is the exam period?",
    "How does Erasmus work?",
    "Where can I find my class schedule?"
]

for q in test_queries:
    print("\nUSER:", q)
    results = chatbot.find_best_answer(q, top_k=3)
    for r in results:
        print(f"  Rank {r['rank']} | Score {r['similarity_score']:.3f} | {r['confidence']}")
        print("   Q:", r["matched_question"])
        print("   A:", r["answer"][:120], "...")
        print("   Category:", r["category"], "| Source:", r["source_url"])



USER: What scholarships can I get?
  Rank 1 | Score 0.469 | High
   Q: What scholarships are available for students?
   A: Warsaw University of Technology offers several types of scholarships: (1) Social scholarship for students whose income p ...
   Category: Scholarships | Source: https://www.il.pw.edu.pl/wp-content/uploads/2024/10/Scholarship-Guide-2024-2025.pdf
  Rank 2 | Score 0.000 | Low
   Q: Co to jest USOSweb?
   A: USOSweb to oficjalny system studencki na PW, który zawiera plan zajęć i egzaminów, przypisanie sal, oceny oraz rejestrac ...
   Category: Campus_Life | Source: https://bwz.uw.edu.pl/en/frequently-asked-questions/
  Rank 3 | Score 0.000 | Low
   Q: Kiedy rozpoczyna się rok akademicki?
   A: Rok akademicki 2025/2026 na Politechnice Warszawskiej rozpoczyna się 01.10.2025 (środa) uroczystą inauguracją roku akade ...
   Category: Academic_Calendar | Source: https://erasmus.pw.edu.pl/erasmusen/Incoming-Students/Academic-calendar

USER: How do I apply for admission?
  Ra

In [5]:
stats = chatbot.get_stats()
print(stats)


{'total_queries': 5, 'faq_size': 44, 'vocabulary_size': 210, 'model_type': 'TF-IDF Cosine Similarity'}


In [6]:
# ==============================
# Interactive chat with chatbot
# ==============================

print("WUT FAQ Chatbot - type 'exit' to quit.\n")
chatbot = FAQChatbot(
    vectorizer=vectorizer,
    question_vectors=question_vectors,
    questions=questions,
    answers=answers,
    categories=categories,
    source_urls=source_urls,
    languages=languages,
)
    
while True:
    user_query = input("You: ").strip()
    if user_query.lower() in ["exit", "quit", "q"]:
        print("Chatbot: Goodbye!")
        break

    results = chatbot.find_best_answer(user_query, top_k=3)

    best = results[0]
    print(f"\nChatbot (best match, {best['confidence']} confidence, score={best['similarity_score']:.3f}):")
    print(best["answer"])
    print(f"\n[Category: {best['category']} | Source: {best['source_url']}]\n")

    print("Other candidates:")
    for r in results[1:]:
        print(f" - Rank {r['rank']} | score={r['similarity_score']:.3f} | {r['category']} | Q: {r['matched_question']}")
    print("\n" + "-"*80 + "\n")


WUT FAQ Chatbot - type 'exit' to quit.




Chatbot (best match, High confidence, score=0.636):
The winter semester classes begin on 02.10.2025 (Thursday). The winter semester runs until 01.02.2026 (Sunday), followed by the winter examination period which starts on 02.02.2026 (Monday).

[Category: Academic_Calendar | Source: https://erasmus.pw.edu.pl/erasmusen/Incoming-Students/Academic-calendar]

Other candidates:
 - Rank 2 | score=0.306 | Academic_Calendar | Q: When is the winter break?
 - Rank 3 | score=0.176 | Academic_Calendar | Q: When does the summer semester start?

--------------------------------------------------------------------------------

Chatbot: Goodbye!


lsngchain
rag agent

or llm like chatgpt etc

In [7]:
# =============================================================================
# WUT Administrative FAQ Chatbot - TF-IDF Baseline (Notebook 2)
#
# Resumen general:
# Este código implementa un chatbot de tipo "retrieval-based" que responde
# preguntas frecuentes (FAQ) sobre la administración de la Warsaw University 
# of Technology (WUT). El sistema no genera texto nuevo, sino que busca en un 
# conjunto de preguntas-respuestas ya preparadas y devuelve la más similar.
#
# Dataset:
# - Fichero: wut_faq_dataset_bilingual.csv
# - Contenido: 44 pares pregunta–respuesta (38 en inglés, 6 en polaco)
# - Campos principales:
#     * question: pregunta del FAQ en lenguaje natural
#     * answer: respuesta oficial basada en fuentes de WUT
#     * category: categoría (Admissions, Scholarships, Exams, Academic_Calendar,
#                 Erasmus, Campus_Life)
#     * language: 'en' o 'pl'
#     * source_url: URL de la fuente oficial (portal estudiantes, Erasmus, etc.)
# - Uso: se cargan todas las preguntas y respuestas como base de conocimiento 
#   para el modelo de recuperación.
#
# Modelo utilizado:
# - Tipo: modelo de recuperación basado en TF-IDF + similitud de coseno
# - Librerías: scikit-learn (TfidfVectorizer, cosine_similarity)
# - Flujo:
#     1) Se vectorizan todas las preguntas del CSV con TfidfVectorizer.
#        * ngram_range=(1, 2) → unigrams y bigrams.
#        * stop_words='english' → elimina stopwords en inglés.
#        * max_features=1000 → limita el tamaño del vocabulario.
#     2) Se calcula la matriz TF-IDF de todas las preguntas del FAQ.
#     3) Cuando el usuario hace una pregunta:
#        * Se vectoriza la query con el mismo vectorizador.
#        * Se calcula la similitud de coseno entre la query y todas las 
#          preguntas del FAQ.
#        * Se ordenan los resultados por similitud y se devuelven los top-k.
# - Clase FAQChatbot:
#     * Almacena el vectorizador, la matriz de preguntas y los arrays de 
#       preguntas, respuestas, categorías, idiomas y URLs.
#     * Método find_best_answer(user_query, top_k):
#           - Devuelve una lista con los mejores matches (rank, score,
#             pregunta coincidente, respuesta, categoría, idioma, URL y 
#             un nivel de confianza High/Medium/Low según el score).
#     * Método get_stats():
#           - Devuelve estadísticas del modelo (nº de queries, tamaño del FAQ,
#             tamaño del vocabulario, tipo de modelo).
#     * Método batch_query():
#           - Permite evaluar varias preguntas a la vez.
#
# Qué hace el notebook paso a paso:
# 1) Carga el CSV wut_faq_dataset_bilingual.csv en un DataFrame de pandas.
# 2) Extrae arrays de preguntas, respuestas, categorías, idiomas y URLs.
# 3) Entrena el vectorizador TF-IDF sobre el texto de las preguntas.
# 4) Construye el objeto FAQChatbot con el vectorizador y la matriz de 
#    preguntas vectorizadas.
# 5) Lanza varias queries de prueba (por ejemplo: "What scholarships can I get?",
#    "How do I apply for admission?", "When is the exam period?",
#    "How does Erasmus work?", "Where can I find my class schedule?") y muestra 
#    para cada una:
#       - La mejor respuesta encontrada (top-1) con score y categoría.
#       - Los candidatos top-3 para análisis.
# 6) Opcionalmente, incluye una celda interactiva tipo bucle while para que el 
#    usuario pueda chatear con el chatbot desde la consola del notebook.
#
# Fortalezas de este enfoque:
# - Muy rápido y ligero (no necesita entrenamiento pesado, solo ajustar TF-IDF).
# - Fácil de reproducir: mismo CSV + mismo código = mismos resultados.
# - Interpretable: se puede inspeccionar la pregunta del FAQ que se ha 
#   considerado más similar.
# - Buen baseline para un Proof of Concept (PoC) de chatbot de FAQs.
#
# Limitaciones y posibles mejoras:
# - TF-IDF es puramente léxico:
#     * No entiende bien sinónimos ni paráfrasis complejas.
#     * Preguntas semánticamente similares pero con palabras diferentes pueden 
#       tener scores bajos.
# - No hay manejo de contexto conversacional:
#     * Cada pregunta se responde de forma independiente, sin tener en cuenta 
#       el historial de la conversación.
# - No hay umbral de confianza/fallback avanzado:
#     * Aunque se etiqueta High/Medium/Low, siempre devuelve algo; se podría 
#       añadir lógica para no responder cuando el score es muy bajo y hacer 
#       fallback (por ejemplo: “No estoy seguro, por favor reformula la pregunta”).
# - Dataset pequeño:
#     * Solo 44 Q&A; en un sistema real haría falta ampliar a 200–250+ pares 
#       con preguntas reales de estudiantes.
# - Mejoras futuras:
#     * Usar modelos semánticos (Sentence-BERT / sentence-transformers) para 
#       captar mejor el significado de las preguntas.
#     * Combinar TF-IDF + embeddings (ensemble) y comparar rendimientos.
#     * Añadir evaluación cuantitativa (precisión@k, NDCG, etc.) con un conjunto 
#       de queries etiquetadas.
#     * Desplegar el modelo como API (FastAPI) o integrar en una interfaz web.
# =============================================================================
